In [1]:
using Pkg
Pkg.activate(".")
Pkg.instantiate()

  Activating project at `~/Progs/CMOE_Gridap_Tutorials`


Explain how to generate a model with cartesian domain

In [27]:
using Gridap
L = 2.0
n = 2
domain = (0,L,0,L)
partition = (n,n)
model = CartesianDiscreteModel(domain,partition)

writevtk(model,"model")

3-element Vector{Vector{String}}:
 ["model_0.vtu"]
 ["model_1.vtu"]
 ["model_2.vtu"]

Explain how to define reference FE and FE spaces

In [28]:
# Solve the Poisson problem
reffe = ReferenceFE(lagrangian,Float64,1)
V = FESpace(model,reffe;conformity=:H1,dirichlet_tags="boundary")
U = TrialFESpace(V,x->x[1])


TrialFESpace()

Explain definition of triangulation and integration measures

In [29]:
Ω = Triangulation(model)
dΩ = Measure(Ω,2)


GenericMeasure()

Explain weak form definition and solution

In [30]:
f(x) = 1.0
a(u,v) = ∫( ∇(v)⋅∇(u) )dΩ
l(v) = ∫( f*v )dΩ
op = AffineFEOperator(a,l,U,V)
uh = solve(op)  

writevtk(Ω,"solution",cellfields=["uh"=>uh])

(["solution.vtu"],)

Display matrices and vector

In [31]:
using SparseArrays
A = get_matrix(op)
b = get_vector(op)
display(Matrix(A))
display(Vector(b))


1×1 Matrix{Float64}:
 2.6666666666666656

1-element Vector{Float64}:
 3.666666666666665

Text explaining basis functions of FE space in physical and reference domain....

In [32]:
using Gridap.FESpaces
using Gridap.CellData
u_basis = get_trial_fe_basis(U) # basis functions of the trial space
u_basis_data = get_data(u_basis)
@show u_basis_data
N1(x) = x[1]*x[2]
N2(x) = x[1]*(1-x[2])
N3(x) = (1-x[1])*x[2]
N4(x) = (1-x[1])*(1-x[2])
ref_FE_basis_funs = [N1,N2,N3,N4]

x1 = L/2 + L/3
x2 = L/3

xp = Point(x1,x2) # Point in the physical cell
@show u_basis(xp)

xp_ref = Point(1/3,1/3) # Point in the reference cell
@show xp_ref

@show ref_FE_basis_xp = [h(xp_ref) for h in ref_FE_basis_funs]
nothing

In [33]:
v_basis = get_fe_basis(V) # basis functions of the test space
@show v_basis(xp)

4-element Vector{Float64}:
 0.11111111111111116
 0.2222222222222222
 0.22222222222222232
 0.4444444444444443

Explanation about FE function -> free degrees of freedom vs fixed degrees of freedom

In [34]:
uhd = zero(U)
uhd_data = get_data(uhd)
@show uhd_data

# u_value_p1 = [1.0, 2.0, 3.0, 4.0]' * ref_FE_basis_xp

4-element Gridap.Arrays.LazyArray{FillArrays.Fill{typeof(Gridap.Fields.linear_combination), 1, Tuple{Base.OneTo{Int64}}}, Gridap.Fields.LinearCombinationField{Vector{Float64}, Gridap.Fields.LinearCombinationFieldVector{Matrix{Float64}, Gridap.Polynomials.MonomialBasis{2, Float64}}}, 1, Tuple{Gridap.Arrays.LazyArray{FillArrays.Fill{Broadcasting{Gridap.Arrays.PosNegReindex{Vector{Float64}, Vector{Float64}}}, 1, Tuple{Base.OneTo{Int64}}}, Vector{Float64}, 1, Tuple{Gridap.Arrays.Table{Int32, Vector{Int32}, Vector{Int32}}}}, FillArrays.Fill{Gridap.Fields.LinearCombinationFieldVector{Matrix{Float64}, Gridap.Polynomials.MonomialBasis{2, Float64}}, 1, Tuple{Base.OneTo{Int64}}}}}:
 LinearCombinationField()
 LinearCombinationField()
 LinearCombinationField()
 LinearCombinationField()

Evaluate weak form using the basis functions

In [35]:
matcontrib = a(u_basis,v_basis)
veccontrib = l(v_basis)

DomainContribution()

In [36]:
data = collect_cell_matrix_and_vector(U,V,matcontrib,veccontrib,uhd)

((Any[[([0.6666666666666664 -0.16666666666666663 -0.16666666666666663 -0.33333333333333315; -0.16666666666666663 0.6666666666666664 -0.33333333333333315 -0.1666666666666666; -0.16666666666666663 -0.33333333333333315 0.6666666666666664 -0.1666666666666666; -0.33333333333333315 -0.1666666666666666 -0.1666666666666666 0.6666666666666663], [0.4166666666666665, -0.4166666666666665, 0.583333333333333, 0.4166666666666665]), ([0.6666666666666664 -0.16666666666666663 -0.16666666666666663 -0.33333333333333315; -0.16666666666666663 0.6666666666666664 -0.33333333333333315 -0.1666666666666666; -0.16666666666666663 -0.33333333333333315 0.6666666666666664 -0.1666666666666666; -0.33333333333333315 -0.1666666666666666 -0.1666666666666666 0.6666666666666663], [0.583333333333333, -0.583333333333333, 1.416666666666666, -0.4166666666666663]), ([0.6666666666666664 -0.16666666666666663 -0.16666666666666663 -0.33333333333333315; -0.16666666666666663 0.6666666666666664 -0.33333333333333315 -0.1666666666666666;

In [37]:
icell = 3
matvec_data = data[1]
celldata = matvec_data[1] # w
rows = matvec_data[2]     # r
cols = matvec_data[3]     # c

println("Cell data: ")
display(length(celldata[1]))
println("Rows: ")
display(rows)
println("Cols: ")
display(cols)

elemental_matrix_icell = Matrix(celldata[1][icell][1])
elemental_vector_icell = Vector(celldata[1][icell][2])
display(elemental_matrix_icell)
display(elemental_vector_icell)

4

1-element Vector{Any}:
 Vector{Int32}[[-1, -2, -4, 1], [-2, -3, 1, -5], [-4, 1, -6, -7], [1, -5, -7, -8]]

1-element Vector{Any}:
 Vector{Int32}[[-1, -2, -4, 1], [-2, -3, 1, -5], [-4, 1, -6, -7], [1, -5, -7, -8]]

4×4 Matrix{Float64}:
  0.666667  -0.166667  -0.166667  -0.333333
 -0.166667   0.666667  -0.333333  -0.166667
 -0.166667  -0.333333   0.666667  -0.166667
 -0.333333  -0.166667  -0.166667   0.666667

4-element Vector{Float64}:
  0.583333333333333
  0.41666666666666646
  0.4166666666666665
 -0.4166666666666664